### Include Librairies

In [22]:
import pandas as pd
import seaborn as sns
import numpy as np
import math
from datetime import *
import pygal             # Usefull for french map
from pygal.style import Style
import matplotlib.pyplot as plt
%matplotlib inline

### Import Data

In [23]:
df = pd.read_csv('data.csv')

In [24]:
df.head()

,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,...,dateFin,dateDebut,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
0,325175,16,32517500016,O,2000-09-26,NaN,NaN,3212ZZ,2015-03-18T00:58:59,False,...,NaN,2009-05-27,F,NaN,NaN,NaN,NaN,32.12Z,NAFRev2,N
1,325175,24,32517500024,O,2008-05-20,NaN,NaN,NaN,2011-12-12T09:40:04,False,...,NaN,2011-10-21,F,TAHITI PERLES CREATIONS,NaN,NaN,NaN,47.89Z,NAFRev2,N
2,325175,32,32517500032,O,2009-05-27,NaN,NaN,NaN,2014-07-08T00:10:21,False,...,NaN,2011-10-21,F,TAHITI PERLES CREATIONS,NaN,NaN,NaN,32.12Z,NAFRev2,N
3,325175,40,32517500040,O,2011-10-21,NaN,NaN,3212ZZ,2015-03-18T00:58:59,False,...,NaN,2014-01-07,F,TAHITI PERLES CREATION,NaN,NaN,NaN,32.12Z,NAFRev2,N
4,325175,57,32517500057,O,2014-01-07,NaN,NaN,NaN,2018-07-10T14:17:15,False,...,2018-02-06,2014-01-07,A,TAHITI PERLES CREATION,NaN,NaN,NaN,32.12Z,NAFRev2,N


### Slipt Columns

In [25]:
columns_int = df.select_dtypes(include=['int']).columns
columns_int

Index(['siren', 'nic', 'siret', 'nombrePeriodesEtablissement',
       'categorieJuridiqueUniteLegale', 'nicSiegeUniteLegale',
       'codePostalEtablissement', 'codeCommuneEtablissement'],
      dtype='object')

In [26]:
columns_float = df.select_dtypes(include=['float']).columns
columns_float

Index(['anneeEffectifsEtablissement', 'denominationUsuelle2UniteLegale',
       'denominationUsuelle3UniteLegale', 'pseudonymeUniteLegale',
       'identifiantAssociationUniteLegale', 'anneeEffectifsUniteLegale',
       'anneeCategorieEntreprise', 'numeroVoieEtablissement',
       'libelleCommuneEtrangerEtablissement', 'codeCedexEtablissement',
       'codePaysEtrangerEtablissement', 'libellePaysEtrangerEtablissement',
       'complementAdresse2Etablissement', 'numeroVoie2Etablissement',
       'indiceRepetition2Etablissement', 'codePostal2Etablissement',
       'libelleCommuneEtranger2Etablissement',
       'distributionSpeciale2Etablissement', 'codeCommune2Etablissement',
       'codeCedex2Etablissement', 'libelleCedex2Etablissement',
       'codePaysEtranger2Etablissement', 'libellePaysEtranger2Etablissement',
       'enseigne2Etablissement', 'enseigne3Etablissement'],
      dtype='object')

In [27]:
columns_object = df.select_dtypes(include=['object']).columns
columns_object

Index(['statutDiffusionEtablissement', 'dateCreationEtablissement',
       'trancheEffectifsEtablissement',
       'activitePrincipaleRegistreMetiersEtablissement',
       'dateDernierTraitementEtablissement', 'etatAdministratifUniteLegale',
       'statutDiffusionUniteLegale', 'unitePurgeeUniteLegale',
       'dateCreationUniteLegale', 'denominationUniteLegale',
       'sigleUniteLegale', 'denominationUsuelle1UniteLegale',
       'sexeUniteLegale', 'nomUniteLegale', 'nomUsageUniteLegale',
       'prenom1UniteLegale', 'prenom2UniteLegale', 'prenom3UniteLegale',
       'prenom4UniteLegale', 'prenomUsuelUniteLegale',
       'activitePrincipaleUniteLegale',
       'nomenclatureActivitePrincipaleUniteLegale',
       'economieSocialeSolidaireUniteLegale', 'caractereEmployeurUniteLegale',
       'trancheEffectifsUniteLegale', 'dateDernierTraitementUniteLegale',
       'categorieEntreprise', 'complementAdresseEtablissement',
       'indiceRepetitionEtablissement', 'typeVoieEtablissement',
   

In [28]:
df['siren'].nunique()

875

In [29]:
df['siren'].count()

1000

In [30]:
df['siret'].nunique()

1000

In [31]:
df.groupby('siren').first()

,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,nombrePeriodesEtablissement,...,dateFin,dateDebut,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
siren,,,,,,,,,,,,,,,,,,,,,
325175,16,32517500016,O,2000-09-26,NaN,NaN,3212ZZ,2015-03-18T00:58:59,False,3,...,2018-02-06,2009-05-27,F,TAHITI PERLES CREATIONS,NaN,NaN,NaN,32.12Z,NAFRev2,N
1807254,14,180725400014,O,1972-05-01,NaN,NaN,NaN,2009-10-02T11:01:38,False,3,...,NaN,2008-01-01,F,NaN,NaN,NaN,NaN,95.24Z,NAFRev2,N
5410220,15,541022000015,O,NaN,NN,NaN,NaN,NaN,True,1,...,NaN,1988-03-31,F,NaN,NaN,NaN,NaN,22.02,NAP,N
5410345,10,541034500010,O,NaN,NaN,NaN,NaN,NaN,True,1,...,NaN,1984-12-25,F,NaN,NaN,NaN,NaN,79.06,NAP,N
5410394,18,541039400018,O,NaN,NN,NaN,NaN,NaN,True,1,...,NaN,1987-12-01,F,NaN,NaN,NaN,NaN,64.42,NAP,N
5410428,14,541042800014,O,1954-01-01,NN,NaN,NaN,NaN,True,1,...,NaN,1989-06-11,F,NaN,NaN,NaN,NaN,70.2C,NAF1993,N
5410436,17,541043600017,O,NaN,NaN,NaN,NaN,NaN,True,1,...,NaN,1984-12-25,F,NaN,NaN,NaN,NaN,57.11,NAP,N
5410485,14,541048500014,O,NaN,NaN,NaN,NaN,NaN,True,1,...,NaN,1985-12-25,F,NaN,NaN,NaN,NaN,64.42,NAP,N
5410501,18,541050100018,O,1954-01-01,NN,NaN,NaN,NaN,True,1,...,NaN,2001-12-31,F,NaN,NaN,NaN,NaN,55.4B,NAF1993,N


In [32]:
df.groupby('siren')['siret'].sum().head()

siren
325175     162587500169
1807254    361450800036
5410220    541022000015
5410345    541034500010
5410394    541039400018
Name: siret, dtype: int64

### Convert Column : Date Creation Etablissement TO Datetime

In [33]:
for i in range(len(df)):
    try:
        math.isnan(df['dateCreationEtablissement'][i]) 
    except:
        df['dateCreationEtablissement'][i] = datetime.strptime(df['dateCreationEtablissement'][i], '%Y-%m-%d')
        pass

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


## NAF APE

### Function Mean Date / Print Info

In [34]:
def mean_date(data):
    tot_date = 0
    count = 0
    
    for value in data:
        try:
            tot_date += value.year
            count += 1
        except:
            pass
    if count != 0:
        return int(tot_date/count)
    else:
        return 0

In [35]:
def print_infos(name_value,nb_value,mean_date_value,name_code,max_code):
    
    total = 0
    
    for i in range(len(name_value)):
        total += nb_value[i]
        try:
            print("-Name : %s, \tnb : %s, mean_date : %s, \tCode Commune : %s, Département : %s,  nb entreprise : %s" %(name_value[i],nb_value[i],mean_date_value[i],name_code[i],code_departement[i],max_code[i]))
        except:
            pass
        
    print("\nTotal siren : ",total)
        

### Find main Region foreach activitie

In [36]:
def max_region(data,df):
    
    list_code_commune = list()
    save_code_commune = list()
    save_count = list()



    for i in range(len(df)):
        try:
            list_code_commune.append(data[i])
        except:
            pass
    
    
    
    for i in range(len(list_code_commune)):
        count = 0
        for j in range(i,len(list_code_commune)):
            if list_code_commune[i] == list_code_commune[j]:
                count += 1
        save_code_commune.append(list_code_commune[i])
        save_count.append(count)
       
    try:
        m = max(save_count)
        m_index = [i for i, j in enumerate(save_count) if j == m]
        name_code = save_code_commune[m_index[0]]
        
    except:
        name_code = 0
        m = 0
        pass
        
    return name_code,m
    

### Function chiffres : Split Code_Commune

In [37]:
def chiffres(n):
    c = []
    while n >= 1:
        c.append(int(n % 10))
        n = n / 10
        
    save = 0
    for i in range(int(len(c)/2)):
        save = c[i]
        c[i] = c[-i -1]
        c[-i-1] = save
    return c

### Convert Code_Commune to Code_Departement

In [38]:
def code_depart(code):
    code_departement = []
    try:
        c = chiffres(int(code))
        #print(c)
        code_departement.append(c[0]*10 +c[1])

    except:
        pass
    
    return code_departement


### Print Info Nomenclature Activite Principale Etablissement

In [39]:
tot = 0
name_value = list()
nb_value = list()
mean_date_value = list()
name_code = list()
max_code = list()
code_departement = list()

col_nomenclature = df['nomenclatureActivitePrincipaleEtablissement'].unique()

for value in col_nomenclature:
    tot_imp = len(df[df['nomenclatureActivitePrincipaleEtablissement'] == value].groupby('siren').first())

    meandate = mean_date(df[df['nomenclatureActivitePrincipaleEtablissement'] == value]['dateCreationEtablissement'])
    code,m = max_region(df[df['nomenclatureActivitePrincipaleEtablissement'] == value]['codeCommuneEtablissement'],df)
    
    
    # Save Infos
    name_value.append(value)
    nb_value.append(tot_imp)
    mean_date_value.append(meandate)
    name_code.append(code)
    max_code.append(m)
    
    try:
        c = chiffres(int(code))
        #print(c)
        code_departement.append(c[0]*10 +c[1])

    except:
        pass
    


# Print infos
print_infos(name_value,nb_value,mean_date_value,name_code,max_code)



-Name : NAFRev2, 	nb : 103, mean_date : 1974, 	Code Commune : 80001, Département : 80,  nb entreprise : 24
-Name : NAP, 	nb : 586, mean_date : 1922, 	Code Commune : 44184, Département : 44,  nb entreprise : 72
-Name : NAF1993, 	nb : 189, mean_date : 1957, 	Code Commune : 4070, Département : 40,  nb entreprise : 24
-Name : NAFRev1, 	nb : 28, mean_date : 1966, 	Code Commune : 80001, Département : 80,  nb entreprise : 8

Total siren :  906


## French map : Use Librairie PYGAL

### Custom Style from map

In [50]:
custom_style = Style(
  background='transparent',
  #plot_background='transparent',
  transition='200ms ease-in',
  #colors=('#e5849b', '#e5849b', '#f3d387', '#8fd9b3', '#E89B53'),
)

### Function which draws chart

In [57]:
def chart():
    fr_chart = pygal.maps.fr.Departments(style=custom_style,height=450)
    fr_chart.title = 'Activities by departments'
    
    for i in range(len(code_departement)):
        fr_chart.add(name_value[i], {code_departement[i]: max_code[i]})
    
    
    #fr_chart.render()           # Exporte to SVG
    fr_chart.render_in_browser() # Exporte to Browser

In [58]:
chart()

file:///var/folders/v3/6rms4ljn2dzdln_l8746trf00000gn/T/tmp4o7gxcv5.html
